1. 前置作業：安裝 & 設定 Groq

In [ ]:
# 安裝相關套件
%pip install -q groq 
%pip install gradio
%pip install tensorflow
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install sklearn
%pip install tensorflowjs
%pip install -q sentence-transformers faiss-cpu
%pip install tf-keras

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from dotenv import load_dotenv
from groq import Groq
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


# 讀取 .env 檔案中的環境變數
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

# 設定 API 金鑰，請自行確認金鑰是否正確
client = Groq(api_key=api_key)

# 在 Colab 中設定你的 GROQ API Key（可放在 Secrets 裡）




2. 定義 Retrieval + Generation 函數（改用 llama-3.3-70b-versatile）

In [3]:
# ── A. 禁用 Transformers TF 支援（保留即可） ───────────────
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# ── B. 下載並解壓 zip（保持不變） ───────────────────────────
import urllib.request, zipfile
url = "https://drive.google.com/uc?export=download&id=1KWwlS5lqqEmiuPNVAw05B76Jyg4JcqAF"
zip_path = "city_guide_dataset.zip"
urllib.request.urlretrieve(url, zip_path)
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("city_data")

# ── C. 用 Scikit-Learn 做 TF-IDF Embedding + 檢索 ────────────
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import glob, os

# 1) 讀入所有 txt 檔
docs, meta = [], []
for fn in glob.glob("city_data/*.txt"):
    city = os.path.splitext(os.path.basename(fn))[0]
    with open(fn, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line: continue
            docs.append(line)
            meta.append(city)

# 2) 建立 TF-IDF 矩陣
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(docs)

# 3) 定義 retrieve
def retrieve(query, top_k=3):
    q_vec = vectorizer.transform([query])
    cosine_similarities = linear_kernel(q_vec, tfidf_matrix).flatten()
    best_idx = cosine_similarities.argsort()[-top_k:][::-1]
    return "\n".join(f"[{meta[i]}] {docs[i]}" for i in best_idx)

from groq import Groq
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def citybot_chat(user_input):
    context = retrieve(user_input, top_k=3)
    system_msg = {"role":"system","content":"你是一個旅遊 AI 諮詢師，請根據檢索到的城市導覽資訊，給使用者完整、詳盡且有禮的建議。"}
    user_msg = {"role":"user","content":f"""== 檢索結果 ==\n{context}\n\n== 使用者提問 ==\n{user_input}\n\n請根據上述資訊作答。"""}
    resp = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[system_msg, user_msg],
        temperature=0.7,
        max_tokens=300
    )
    return resp.choices[0].message.content.strip()


3. Gradio 介面

In [4]:
import gradio as gr

iface = gr.Interface(
    fn=citybot_chat,
    inputs=gr.Textbox(lines=2, placeholder="輸入，例如：我想去看古蹟，推薦哪個城市？"),
    outputs="text",
    title="🏙️ CityBot 旅遊向導（Groq + Llama）",
    description="基於 RAG 檢索及 Groq Llama-3.3-70b-versatile，給你最合適的城市旅遊建議！"
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://92aebdfd481231c113.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
